In [ ]:
%%file noise.py

import numpy as np
import matplotlib.pyplot as plt

import pyNN.hardware.spikey as pynn

N_NEURONS = 32

input_rates = np.linspace(0.0, 1000.0, 10.0)
output_rates = np.zeros((N_NEURONS, input_rates.size))

for i, input_rate in enumerate(input_rates):
    pynn.setup(mappingOffset=0, calibSynDrivers=True)

    stimulus = pynn.Population(1, pynn.SpikeSourcePoisson, {'rate': input_rate})

    neurons = pynn.Population(N_NEURONS, pynn.IF_facets_hardware1)
    neurons.record()

    weight = 15 * pynn.minExcWeight()

    proj_stim = pynn.Projection(stimulus, neurons,
           pynn.AllToAllConnector(weights=weight),
           target="excitatory")

    pynn.run(1000.0)
    
    spikes = neurons.getSpikes()
    
    pynn.end()
    
    for nrn in np.unique(spikes[:,0]):
        mask = (spikes[:,0] == nrn)
        output_rates[int(nrn),i] = spikes[mask,1].size

fig = plt.figure(figsize=(12, 6))
ax = fig.gca()

ax.set_xlabel(r"time [\si{\milli\second}]")
ax.set_ylabel(r"firing rate [\si{\hertz}]")

for nrn in xrange(N_NEURONS):
    ax.plot(input_rates, output_rates[nrn,:], ".-")

fig.savefig("noise.png")

In [ ]:
from IPython.display import Image
! srun --gres=station508 python noise.py
Image(filename="noise.png")